In [ ]:
# !pip install "autogen-ext[grpc]"

In [20]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start()  # Start a host service in the background.

In [21]:
from dataclasses import dataclass

from autogen_core import DefaultTopicId, MessageContext, RoutedAgent, default_subscription, message_handler


@dataclass
class MyMessage:
    content: str


@default_subscription
class MyAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__("My agent")
        self._name = name
        self._counter = 0

    @message_handler
    async def my_message_handler(self, message: MyMessage, ctx: MessageContext) -> None:
        self._counter += 1
        # if self._counter > 5:
        #     return
        content = f"{self._name}: Hello x {self._counter}"
        print(content)
        # await self.publish_message(MyMessage(content=content), DefaultTopicId())

In [14]:
import asyncio

from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
worker1.start()
await MyAgent.register(worker1, "worker1", lambda: MyAgent("worker1"))


AgentType(type='worker1')

In [22]:

worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
worker2.start()
await MyAgent.register(worker2, "worker2", lambda: MyAgent("worker2"))




AgentType(type='worker2')

In [24]:

await worker2.publish_message(MyMessage(content="Hello There"), DefaultTopicId())

# Let the agents run for a while.
await asyncio.sleep(2)

In [17]:
await worker1.stop()


In [18]:
await worker2.stop()

# To keep the worker running until a termination signal is received (e.g., SIGTERM).
# await worker1.stop_when_signal()

In [19]:
await host.stop()

# To keep the host service running until a termination signal (e.g., SIGTERM)
# await host.stop_when_signal()